# **MATH&ML-2. Линейная алгебра в контексте линейных методов. Часть II**

# 1. Введение

✍ Рады приветствовать вас во втором модуле, посвящённом линейной алгебре!

В предыдущем модуле мы познакомились с базовыми понятиями линейной алгебры. По сути, мы изучили основы матричного языка и теперь можем на нем говорить. 

В этом модуле мы перейдём к применению линейной алгебры в машинном обучении и рассмотрим **алгоритмы анализа данных** с разных сторон. В основном мы, конечно, будем говорить о модели линейной регрессии и её модификациях.

Мы будем использовать весь математический аппарат, который изучили в прошлом модуле. Темы предстоят очень интересные, но в то же время непростые. Для того чтобы усвоить их, необходимо владеть всеми навыками, приобретёнными в предыдущем модуле — от базовых операций над векторами до знания принципов решения СЛАУ. Давайте проверим, насколько вы готовы ↓

### Задание 1.1

Найдите скалярное произведение векторов:

$\vec{v_{1}}  = (-1, 2, \ -7, 9)^T$

$\vec{v_{2}}  = (2, 8, 2, \ -1)^T$

In [2]:
import numpy as np

np.array([-1, 2, -7, 9])@np.array([2, 8, 2, -1])

-9

In [5]:
np.array([[1, 1], [1, 2]])@np.array([[1, 1], [1, 2]])

array([[2, 3],
       [3, 5]])

### Задание 1.4

Задана матрица

![](https://lms.skillfactory.ru/assets/courseware/v1/c1579bdd2592b86d75c9413c69d0627d/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_1_1.png)

Найдите матрицу Грама $(A^{T}A)$.

In [15]:
A = np.array([[1, 1], [1, 2]])
print('(A^T)*A: \n', A.T@A)


(A^T)*A: 
 [[2 3]
 [3 5]]


### Задание 1.6

Вычислите обратную матрицу $A^{-1}$, если:

![](https://lms.skillfactory.ru/assets/courseware/v1/e9a41d606d7dcf555779b45d871b27ce/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_1_4.png)

In [17]:
A = np.array([[1, 2], [3, 7]])
np.linalg.inv(A)

array([[ 7., -2.],
       [-3.,  1.]])

### Задание 1.7

Найдите ранг матрицы системы, составленной из векторов:

$\vec{v_{1}}  = (2, 10, \ -2)^T$

$\vec{v_{2}}  = (3, 2, \ -2)^T$

$\vec{v_{3}}  = (8, 14, \ -6)^T$

In [19]:
v1 = np.array([2, 10, -2])
v2 = np.array([3, 2, -2])
v3 = np.array([8, 14, -6])

A = np.array([v1, v2, v3]).T
np.linalg.matrix_rank(A)

2

**В первой части** мы будем говорить о **классической модели линейной регрессии**. Для этого мы вернёмся к неоднородным системам линейных алгебраических уравнений, которые мы затронули в прошлом модуле, и посмотрим, как они связаны с **методом наименьших квадратов** (МНК, или OLS, Ordinary Least Squares). Затем мы с математической точки зрения посмотрим на проблемы, которые возникают при его использовании, например мультиколлинеарность или чересчур большое количество факторов.

**Во второй части** мы перейдём к модификациям модели линейной регрессии и посмотрим, как линейная алгебра работает в **полиномиальной регрессии**, а также поговорим о том, как работают методы регуляризации на математическом уровне.

В конце каждого из блоков нас ожидает небольшая **практическая задача** на применение регрессионных моделей.

Сразу отметим, что **мы не будем рассматривать следующие вопросы**:

* Вероятностные предпосылки использования модели линейной регрессии, необходимые для её валидности (данные предпосылки регламентирует теорема Маркова-Гаусса).
* Оценки качества полученной регрессионной модели и оценки статистической значимости её коэффициентов.
* Статистические методы предварительной обработки данных.

Для изучения этих вопросов понадобятся знания в области теории вероятности и статистики. Мы обязательно обсудим их в следующих модулях. А пока нас интересуют подробности построения вычислительных алгоритмов с точки зрения линейной алгебры.

* Также мы не будем затрагивать тему корректной валидации моделей и разделять выборку на тренировочную/тестовую/валидационную с целью экономии времени и сил. Надеемся, что вы помните, как правильно оценивать качество получаемых моделей, сможете проделать это самостоятельно.

Таким образом, мы будем рассматривать **только математическую составляющую** линейных (и полиномиальных) моделей регрессии, не концентрируясь на сторонних аспектах машинного обучения.

## ЦЕЛИ ДАННОГО МОДУЛЯ:

* познакомиться с неоднородными СЛАУ и случаями их решений;
* изучить математическую формализацию метода наименьших квадратов;
* научиться строить модель линейной регрессии с помощью МНК;
* понять, какие проблемы возникают в МНК с математической точки зрения;
* познакомиться с математической формализацией полиномиальной регрессии;
* рассмотреть методы регуляризации и принципы их работы.

# 2. Неоднородные СЛАУ

✍ Мы начнём с **алгоритма классической линейной регрессии по методу наименьших квадратов** (OLS, Ordinary Least Squares). Данный алгоритм является базовым, но, тем не менее, весьма непрост для восприятия, поэтому данная сложносочинённая задача будет разделена на две части:

* В этом юните мы обсудим случаи и алгоритм решения неоднородных СЛАУ.
* В следующем юните подведём под эту задачу контекст задачи регрессии.

Для начала давайте вспомним, что такое **неоднородные СЛАУ**.

**Примечание**. Совокупность уравнений первой степени, в которых каждая переменная и коэффициенты в ней являются вещественными числами, называется **системой линейных алгебраических уравнений** (**СЛАУ**) и в общем случае записывается как:

$$\left\{ \begin{array}{c} a_{11}x_1+a_{12}x_2+\dots +a_{1m}x_m=b_1 \\ a_{21}x_1+a_{22}x_2+\dots +a_{2m}x_m=b_2 \\ \dots \\ a_{n1}x_1+a_{n2}x_2+\dots +a_{nm}x_m=b_n \end{array} \right.\ (1),$$

где

* $n$ — количество уравнений;
* $m$ — количество переменных;
* $x_i$ — неизвестные переменные системы;
* $a_{ij}$ — коэффициенты системы;
* $b_i$ — свободные члены системы.

CЛАУ (1) называется **однородной**, если все свободные члены системы равны 0 ($b_1=b_2=⋯=b_n=0$):

$$\textrm{С}\textrm{Л}\textrm{А}\textrm{У}-\textrm{о}\textrm{д}\textrm{н}\textrm{о}\textrm{р}\textrm{о}\textrm{д}\textrm{н}\textrm{а}\textrm{я},\ \textrm{е}\textrm{с}\textrm{л}\textrm{и}\ \forall b_i=0$$

СЛАУ (1) называется **неоднородной**, если хотя бы один из свободных членов системы отличен от 0:

$$\textrm{С}\textrm{Л}\textrm{А}\textrm{У}-\textrm{н}\textrm{е}\textrm{о}\textrm{д}\textrm{н}\textrm{о}\textrm{р}\textrm{о}\textrm{д}\textrm{н}\textrm{а}\textrm{я},\ \textrm{е}\textrm{с}\textrm{л}\textrm{и}\ \exists b_i\neq 0$$

Пример неоднородной СЛАУ:

$\left\{\begin{array}{c} w_{1}+w_{2}=1 \\ w_{1}+2 w_{2}=2 \end{array}\right.$

Вспомним, что СЛАУ можно записать в матричном виде:

![](https://lms.skillfactory.ru/assets/courseware/v1/35808bc9ca767f3c996106d8887c1108/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_3.png)

где $A$ — матрица системы, $\omega$ — вектор неизвестных коэффициентов, а $b$ — вектор свободных членов. 

Давайте введём новое для нас определение.

> Р**асширенной матрицей системы $A|b$ неоднородных СЛАУ** называется матрица, составленная из исходной матрицы и вектора свободных коэффициентов:
>
> $$(A \mid \vec{b})=\left(\begin{array}{cccc|c} a_{11} & a_{12} & \ldots & a_{1 m} & b_{1} \\ a_{21} & a_{22} & \ldots & a_{2 m} & b_{2} \\ \ldots & \ldots & \ldots & \ldots & \ldots \\ a_{n 1} & a_{n 2} & \ldots & a_{n m} & b_{n} \end{array}\right)$$

Расширенная матрица системы — это обычная матрица. Черта, отделяющая коэффициенты $a_{ij}$ от свободных членов $b_i$ — чисто символическая. 

Над расширенной матрицей неоднородной СЛАУ можно производить те же самые действия, что и над обычной, а именно:

* складывать/вычитать между собой строки/столбцы матрицы;
* умножать строки/столбцы на константу;
* менять строки/столбцы местами.

Приведём пример расширенной матрицы системы. Пусть исходная система будет следующей:

$\left\{\begin{array}{c} w_{1}+w_{2}=1 \\ w_{1}+2 w_{2}=2 \end{array}\right.$

Запишем её в матричном виде:

![](https://lms.skillfactory.ru/assets/courseware/v1/f928ba511cc7a2ac62881707fdfeea66/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_6.png)

Тогда расширенная матрица системы будет иметь вид:

![](https://lms.skillfactory.ru/assets/courseware/v1/f534ccbdd7c85b138e033bbb178a520b/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_7.png)

еперь, когда мы вспомнили все определения и познакомились с термином расширенной матрицы, мы можем переходить к решению неоднородных СЛАУ.

Существует три случая при решении неоднородных СЛАУ:

* «Идеальная пара»

    Это так называемые определённые системы линейных уравнений, имеющие единственные решения.

* «В активном поиске»

    Неопределённые системы, имеющие бесконечно много решений.

* «Всё сложно»

    Это самый интересный для нас случай — переопределённые системы, которые не имеют точных решений.

**Примечание**. В данной классификации неоднородных СЛАУ допущено упрощение в терминологии. На самом деле неопределённые системы — это те, в которых независимых уравнений меньше, чем неизвестных. Они могут иметь бесконечно много решений (быть совместными) или ни одного решения (быть несовместными, если уравнения противоречат друг другу).

На практике, например в обучении регрессий, этот случай практически не встречается.

Что касается переопределённых систем, то в них, помимо несовместности (отсутствия решений), количество независимых уравнений превышает количество неизвестных — это тот самый случай, что мы видим в регрессионном анализе.

Далее мы рассмотрим каждый из случаев на примере.

## СЛУЧАЙ «ИДЕАЛЬНАЯ ПАРА»

> Самый простой случай решения неоднородной СЛАУ — когда система имеет единственное решение. Такие системы называются **совместными**.

На вопрос о том, когда СЛАУ является совместной, отвечает главная теорема СЛАУ — **теорема Кронекера — Капелли** (также её называют **критерием совместности системы**).

> **Теорема Кронекера — Капелли:**
> 
> Неоднородная система линейный алгебраических уравнений $A \vec{w} = \vec{b}$ является совместной тогда и только тогда, когда ранг матрицы системы $A$ **равен** рангу расширенной матрицы системы $(A| \vec{b})$ и равен количеству независимых переменных $m$:
> 
> $$rk(A) = rk(\vec{b}) = m \leftrightarrow \exists ! \vec{w} = (w_{1}, w_{2}, \ldots w_m)^T$$
>
> Причём решение системы будет равно:
> 
> $$\vec{w} = A^{-1} \vec{b}$$


**Примечание**. Здесь значок $\exists !$ переводится как «существует и причём единственное».

Сложно и непонятно? Давайте разберёмся, как работает эта теорема, на примерах ↓

*Дана СЛАУ:*

![](https://lms.skillfactory.ru/assets/courseware/v1/cadd1684cd0b6d152ce5a1a784fc8973/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_8.png)

*где $w_1$ и $w_2$ — неизвестные переменные.*

При решении системы «в лоб» получим:

![](https://lms.skillfactory.ru/assets/courseware/v1/37c7d45c0a88651cd57dd05b085adb78/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_9.png)

Интерпретация:

![](https://lms.skillfactory.ru/assets/courseware/v1/5191ad698399cde0da309f8deea15cb6/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_10.png)

На языке линейной алгебры это означает что вектор $(1,2)^T$ линейно выражается через векторы коэффициентов системы $(1,1)^T$ и $(1,2)^T$.

В матричном виде система запишется, как:

![](https://lms.skillfactory.ru/assets/courseware/v1/a1e86f40406ca7904279ac19a60844b0/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_11.png)

Преобразование уравнений будем таким же, как и при преобразовании расширенной матрицы системы $(A|\vec{b})$, вычитая сначала первую строку из второй, а затем — результат из первой, получим то же решение, что и решение «в лоб».

![](https://lms.skillfactory.ru/assets/courseware/v1/d0df10f19deeb809ba7cbf27264f85a9/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_12.png)

Других решений у системы нет. 

Посмотрим на ранги матрицы $A$ и расширенной матрицы $(A|\vec{b})$ (количество ступеней в ступенчатых матрицах):

![](https://lms.skillfactory.ru/assets/courseware/v1/da90d08d05f922039ad20d58aa5aff45/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_13.png)

Они совпадают и равны количеству неизвестных, а это и гарантирует существование и единственность решения. То есть в общем случае, чтобы узнать, сколько решений существует у системы, её необязательно было бы решать — достаточно было бы найти ранги матриц $A$ и $A|b$.

Тут возникает вопрос: «Можно ли найти решение одной формулой?»

Для удобства перепишем систему без стрелок:

$Aw = b$

Так как матрица квадратная и невырожденная, у неё обязательно есть обратная матрица.

Умножим на $A^{-1}$ слева обе части уравнения. Стоит напомнить, что произведение матриц не перестановочно, поэтому есть разница, с какой стороны умножать.

$$A^{-1} Aw = A^{-1}b$$

$$w = A^{-1} b$$


**Важно**! Отсюда явно видны **ограничения** этого метода: его можно применять только для квадратных невырожденных матриц (тех, у которых определитель не равен 0).

Убедимся в правильности формулы. Найдём произведение матрицы $A^{-1}$ и вектора-столбца $b$:

![](https://lms.skillfactory.ru/assets/courseware/v1/207d07ac7518f16e548924a9bc2e3382/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_14.png)

Меняем местами главную диагональ и меняем знаки у побочной. Как видим, результат совпал с прошлым методом.

**Резюмируем ↓**

У нас есть квадратная система с $m$ неизвестных. Если ранг матрицы коэффициентов $A$ **равен** рангу расширенной матрицы $(A|b)$ и **равен** количеству переменных ($rk(A)=rk(\vec{b})=m$), то в системе будет ровно столько независимых уравнений, сколько и неизвестных $m$, а значит будет **единственное** решение.

Вектор свободных коэффициентов $b$ при этом линейно независим со столбцами матрицы $A$, его разложение по столбцам $A$ единственно.

### Задание 2.4

*Решите систему линейных уравнений:*

![](https://lms.skillfactory.ru/assets/courseware/v1/bb410f21af5bc73df165cf79d3623d37/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_23.png)

In [45]:
A = np.array([[4, 7], [5, 10]])
b = np.array([20, 30])
A_inv = np.linalg.inv(A)
print('[x y] = ', A_inv@b)

[x y] =  [-2.  4.]


## СЛУЧАЙ «В АКТИВНОМ ПОИСКЕ»

А что, если система не удовлетворяет теореме Кронекера — Капелли? То есть ранг матрицы системы равен расширенному рангу матрицы, но не равен количеству неизвестных. Неужели тогда система нерешаема?

На этот вопрос отвечает первое следствие из теоремы ↓

> **Следствие №1 из теоремы Кронекера — Капелли:**
> 
> Если ранг матрицы системы $A$ равен рангу расширенной матрицы системы $(A|\vec{b})$, но меньше, чем количество неизвестных $m$, то система имеет бесконечное множество решений:
> 
> $$rk(A) = rk(A | \vec{b}) < m  \leftrightarrow  \infty \ решений$$

Вновь рассмотрим пример.

*Решим систему уравнений:*

$w_1 + w_2 + w_3 = 10$

Да, уравнение одно, но формально оно является неоднородной СЛАУ.

Итак, мы имеем одно уравнение на три неизвестных, значит две координаты из трёх вектора $w$ мы можем задать как угодно. Например, зададим вторую и третью как $\alpha$ и $\beta$. Тогда первая будет равна $10 - \alpha - \beta$.

![](https://lms.skillfactory.ru/assets/courseware/v1/426d121f84b92a0d96b78bd777ebd44d/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_30.png)

Вместо переменных $\alpha$ и $\beta$ мы можем подставлять любые числа и всегда будем получать равенство. 

Составим расширенную матрицу:

![](https://lms.skillfactory.ru/assets/courseware/v1/81cc95b153a93a58dc3c517470779345/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_31.png)

Её ранг, как и ранг $A$, равен 1, что меньше числа неизвестных $m=3$:

$rk(A) = rk(A | \vec{b}) = 1 < 3$

Такая ситуация, по следствию из теоремы Кронекера — Капелли, говорит о существовании и не единственности решения, то есть решений бесконечно много.




**Резюмируем ↓**

Если ранги матриц $A$ и $(A|\vec{b})$ всё ещё совпадают, но уже меньше количества неизвестных ($rk(A) = rk(A | \vec{b}) < m$), значит, уравнений не хватает для того, чтобы определить систему полностью, и решений будет бесконечно много.

На языке линейной алгебры это значит, что вектор $\vec{b}$ линейно зависим со столбцами матрицы $A$, но также и сами столбцы зависимы между собой, поэтому равнозначного разложения не получится, т. е. таких разложений может быть сколько угодно.

## СЛУЧАЙ «ВСЁ СЛОЖНО»

А теперь посмотрим на самый интересный для нас случай. Его формально регламентирует второе следствие из теоремы Кронекера — Капелли.

> Следствие №2 из теоремы Кронекера — Капелли:
> 
> Если ранг матрицы системы $A$ меньше, чем ранг расширенной матрицы системы $(A|\vec{b})$, то система несовместна, то есть не имеет точных решений:
> 
> $$rk(A)  < rk(A | \vec{b})  \leftrightarrow  \nexists \ решений$$

*Решим систему уравнений:*

![](https://lms.skillfactory.ru/assets/courseware/v1/abcb754bb9f9181e93b95b49e77ff25c/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_42.png)

Посмотрим на первое и третье уравнение — очевидно, что такая система не имеет решений, так как данные уравнения противоречат друг другу.

Но давайте обоснуем это математически. Для этого запишем расширенную матрицу системы:

![](https://lms.skillfactory.ru/assets/courseware/v1/9f9d4dbee7564d18fa820c0a54a3785e/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_43.png)

Посчитаем ранги матриц $A$ и $A|\vec[b$:

![](https://lms.skillfactory.ru/assets/courseware/v1/2774b7f6264958ea7c7d6f23df959a77/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_44.png)

Итак, $rk(A)=2$, в то время как $rk(A|\vec(b))=3$. Это и есть **критерий переопределённости** системы уравнений: ранг матрицы системы меньше ранга расширенной матрицы системы.

→ Получается, что идеальное решение найти нельзя, но чуть позже мы увидим, что такие системы возникают в задачах регрессии практически всегда, а значит нам всё-таки хотелось бы каким-то образом её решать. Можно попробовать найти приблизительное решение — вопрос лишь в том, какое из всех этих решений лучшее.

?

Найдем наилучшее приближение для $w_1$, $w_2$, если:

![](https://lms.skillfactory.ru/assets/courseware/v1/4267dd365e0b367f66729d1cf0eb4b5e/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_45.png)

Обозначим приближённое решение как $\hat{w}$. Приближением для вектора $b$ будет $\hat{b}=A\hat{w}$. Также введём некоторый вектор ошибок $e = b - \hat{b} = b - A \hat{w}$.

**Примечание**. Здесь мы снова опустили стрелки у векторов $b$, $\hat{b}$ и $\hat{w}$ для наглядности.

Например, если мы возьмём в качестве вектора $\hat{w}$ вектор $\hat{w}_1=(1, 1)^T$, то получим:

![](https://lms.skillfactory.ru/assets/courseware/v1/a9f15e57bf47fd7db7b10f9986d878a1/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_46.png)

Теперь возьмём в качестве вектора $\hat{w}_2 = (4, -1)^T$, получим:

![](https://lms.skillfactory.ru/assets/courseware/v1/cc797ce965e4cfa42340afcc97494476/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_47.png)

→ Конечно, нам хотелось бы, чтобы ошибка была поменьше. Но какая из них поменьше? Векторы сами по себе сравнить нельзя, но зато можно сравнить их длины.

Для первого случая будем иметь:

$\left\|e_1 \right\| = \sqrt{(-1)^2 + (-1)^2 + (10)^2} = \sqrt{102}$

Для второго случая:

$\left\|e_2 \right\| = \sqrt{(-2)^2 + 0^2 + 9^2} = \sqrt{85}$

Видно, что вторая ошибка всё-таки меньше, соответственно, приближение лучше. Но в таком случае из всех приближений нам нужно выбрать то, у которого длина вектора ошибок минимальна, если, конечно, это возможно.

$\left\|e \right\| \rightarrow min$

Вернёмся к задаче поиска оптимальных приближений вектора $\hat{w}$.

**Примечание**. Проблема поиска оптимальных приближённых решений неоднородных переопределённых СЛАУ стояла у математиков вплоть до XIX века. До этого времени математики использовали частные решения, зависящие от вида уравнений и размерности. Впервые данную задачу для общего случая решил Гаусс, опубликовав метод решения этой задачи, который впоследствии будет назван методом наименьших квадратов (МНК). В дальнейшем Лаплас прибавил к данному методу теорию вероятности и доказал оптимальность МНК-оценок с точки зрения статистики.

Сейчас мы почувствуем себя настоящими математиками и попробуем решить эту задачу самостоятельно с помощью простой геометрии и знакомых нам операций над матрицами.

Вспомним, что на языке линейной алгебры неразрешимость системы

![](https://lms.skillfactory.ru/assets/courseware/v1/02f4b4ff440bbe548de99fd21d758ef2/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_48.png)

означает, что попытка выразить вектор $(1, 2, 12)^T$ через $(1, 1, 1)^T$ и $(1,2,1)^T$ не будет успешной, так как они линейно независимы.

**Геометрически** это означает, что вектор свободных коэффициентов  (коричневый) не лежит в одной плоскости со столбцами матрицы  (синие векторы).

![](https://lms.skillfactory.ru/assets/courseware/v1/2448deb8bce151ef64ddb640a1fa57db/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_49.png)

→ Напомним, что подобную задачу мы решали в предыдущем модуле по линейной алгебре, в юните «Практика: векторы». Вы можете вернуться в предыдущий модуль и освежить в памяти решение задачи.

Идея состояла в том, что наилучшим приближением для коричневого вектора будет ортогональная проекция на синюю плоскость — голубой вектор. Так происходит потому, что наименьший по длине вектор ошибок — красный — должен быть перпендикулярен к синей плоскости:

$$e=b=\hat{b}$$

В прошлом модуле мы производили расчёты интуитивно, а теперь настала пора вывести формулу.

Давайте умножим наши уравнения слева на $A^T$:

![](https://lms.skillfactory.ru/assets/courseware/v1/f6cba5210f0383927c24de2da993db79/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_50.png)

Идея заключается в следующем: справа мы найдём скалярное произведение столбцов матрицы $A$ на вектор $b$, а слева — произведение столбцов $A$ на приближённый вектор $\hat{b}$ (по сути, на голубую проекцию).

![](https://lms.skillfactory.ru/assets/courseware/v1/04f3d1836f3f2e04db6db6dad688862a/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_71.png)

Упростим уравнение, перемножив всё, что содержит только числа. В левой части умножим $A^T$ на $A$, в правой — умножим $A^T$ на $b$. Тогда слева получим матрицу 2×2 — это не что иное, как матрица Грама столбцов $A$.

Столбцы $A$ линейно независимы, а это значит, что, по свойству матрицы Грама, $A^T \cdot A$  — невырожденная квадратная матрица (её определитель не равен нулю, и для неё существует обратная матрица). Получившаяся система — один в один случай «идеальная пара», а это значит, что теперь мы можем её решить.

![](https://lms.skillfactory.ru/assets/courseware/v1/227fe95c6711c25be991e9c81e9c0d93/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_51.png)

?
Но ведь мы не могли решить изначальную задачу, так как она была переопределена, а эту — можем. Как так получилось?

Мы потребовали, чтобы у приближения $\hat{b}$ были с векторами $(1,1,1)^T$ и $(1,2,1)^T$ такие же скалярные произведения, как у $b$. Это и означает что $\hat{b}$ — ортогональная проекция на нашу синюю плоскость, в которой лежат столбцы матрицы $A$, и в этой плоскости мы можем найти коэффициенты.

Мы с вами отлично умеем решать системы типа «Идеальная пара». Для этого нам нужно найти обратную матрицу  и умножить на неё слева всё уравнение. Так мы и получим наше приближение:

![](https://lms.skillfactory.ru/assets/courseware/v1/926b933be4ea80370273f21776528ca2/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_52.png)

Находим определитель матрицы $(A^TA)$:

$det(A^T A) = 3 \cdot 6 - 4 \cdot 4 = 2$

Находим обратную матрицу (A^TA)^{-1}$:

![](https://lms.skillfactory.ru/assets/courseware/v1/0a77b6e3a1603d68db6cda0ebca99758/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_53.png)

Умножаем всё уравнение на обратную матрицу слева:

![](https://lms.skillfactory.ru/assets/courseware/v1/72ad70cfed4736faeba2e57ace2fa47d/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_54.png)

И, наконец, вот он — долгожданный приближённый вектор $\hat{w}$:

![](https://lms.skillfactory.ru/assets/courseware/v1/456b762ff60c55233bd071d5da87ebfd/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_55.png)

Ещё раз посмотрим на финальную формулу:

![](https://lms.skillfactory.ru/assets/courseware/v1/d22d5f53531f1b98a5c2608008f670dd/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_56.png)

⭐ Пришло время открытий!

Только что мы геометрическим образом вывели формулу оценки решения методом наименьших квадратов (МНК или OLS, Ordinary Least Squares).

**Примечание**. Стоит отметить, что полученная матричная формула не зависит от размерностей и конкретных значений, а значит применима не только в нашем локальном случае, но и в общем.

Нам осталось выполнить проверку полученных результатов, чтобы убедиться в верности решения.

Вычислим голубой вектор $\hat{b}$. Для этого возьмём линейную комбинацию столбцов матрицы $А$ с найденными нами коэффициентами $\hat{w_1}$ и $\hat{w_2}$:

![](https://lms.skillfactory.ru/assets/courseware/v1/18d29c611a2228d28dfd178039e199b0/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_57.png)

Вычислим вектор ошибок $e$:

![](https://lms.skillfactory.ru/assets/courseware/v1/1674251d755c8b031c983072f6557e00/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_58.png)

Убедимся, что данный вектор действительно ортогонален столбцам матрицы $А$. Для этого найдём их скалярные произведения:

![](https://lms.skillfactory.ru/assets/courseware/v1/583371dd14e680cce62a7c551fe859ee/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_59.png)

![](https://lms.skillfactory.ru/assets/courseware/v1/dffc86afc7e86faa61d314c567027151/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_60.png)

Скалярные произведения равны 0, а это означает, что вектор ошибок $e$ действительно ортогонален всей синей плоскости, а голубой вектор $b$ приближённого значения является ортогональной проекцией коричневого вектора $b$.

> **Примечание**. Прежде чем перейти к выводам, стоит отметить, что обычно OLS-оценку выводят немного иначе, а именно минимизируя в явном виде длину вектора ошибок по коэффициентам $\hat{w}$, вернее, даже квадрат длины для удобства вычисления производных.
> 
> $$\left\|\vec{e} \right\| \rightarrow min$$
> 
> $$\left\|\vec{e} \right\|^2 \rightarrow min$$
> 
> $$\left\|\vec{b} - A \vec{w} \right\|^2 \rightarrow min$$
> 
> Формула получится точно такой же, какая есть у нас, просто способ вычислений будет не геометрический, а аналитический. Мы вернёмся к этому способу, когда будем обсуждать оптимизацию функции многих переменных в разделе по математическому анализу.
> 
> Наконец, мы может подвести итоги для случая «Всё сложно».

**Резюмируем ↓**

Если ранг матрицы $A$ меньше ранга расширенной системы $(A|\vec{b})$, то независимых уравнений больше, чем переменных $(rkA<(A|\vec{b})<m)$, а значит некоторые из них будут противоречить друг другу, то есть решений у системы нет.

Говоря на языке линейной алгебры, вектор $b$ линейно независим со столбцами матрицы $A$, а значит его нельзя выразить в качестве их линейной комбинации.

Однако можно получить приближённые решения по методу наименьших квадратов ($OLS-оценка - \hat{b} = (A^{T}A)^{-1}\cdot A^{T} b$), идеей которого является ортогональная проекция вектора  на столбцы матрицы $A$.

### Задание 2.9

Вычислите вектор ошибок для приближённого решения системы , если:

![](https://lms.skillfactory.ru/assets/courseware/v1/e5455f82f2c719a89defd6fbf935bf99/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_61.png)

In [85]:
A = np.array([[1,-5],[2,1],[1,1]])
b = np.array([1,2,2])
w_hat = np.array([1,1])
e = b - A@w_hat
e

array([ 5, -1,  0])

### Задание 2.11

*Найдите $OLS$-оценку для коэффициентов $w_1$, $w_2$ СЛАУ:*

![](https://lms.skillfactory.ru/assets/courseware/v1/aabe72a5dc4493bf7f00fce4be816414/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md2_2_63.png)

*Для этого выполните задания под цифрами 1-4 ниже.*



In [89]:
A = np.array([[1,2],[-3,1],[1,2],[1,-1]])
b = np.array([1,4,5,0])
print(' A : \n',A, '\n', 'b : ', b)

 A : 
 [[ 1  2]
 [-3  1]
 [ 1  2]
 [ 1 -1]] 
 b :  [1 4 5 0]


1. Вычислите матрицу Грама столбцов $A:A^{T}A$

In [90]:
A.T@A

array([[12,  0],
       [ 0, 10]])

2. Вычислите матрицу $(A^{T}A)^{-1}$. Она имеет вид:

In [91]:
np.linalg.inv(A.T@A)

array([[0.08333333, 0.        ],
       [0.        , 0.1       ]])

3. Вычислите $A^{T} \vec{b}$. Введите координаты полученного вектора через запятую, без пробелов. Пример ввода ответа: 1,1.

In [92]:
A.T@b

array([-6, 16])

4. Вычислите вектор оценок коэффициентов $\vec{v}$.

Примечание: для корректного ответа не округляйте $(A^TA)^{-1}$.

Чему равен полученный вектор коэффициентов?

In [93]:
w_hat = np.linalg.inv(A.T@A)@A.T@b
w_hat

array([-0.5,  1.6])

# 3. Линейная регрессия по методу наименьших квадратов
